# Proyecto Matematicas Discretas
## Mejores localizaciones para los centros de atención a mujeres víctimas de violencia en la CDMX
### Yañez Espindola Jose Marcos

### Lipieza y seleccion de los datos

In [1]:
import pandas as pd 
import numpy as np 
import networkx as nx

In [2]:
datos=pd.read_csv('denuncias-victimas-pgj.csv')
datos

,id,idcarpeta,delito,categoria,fechahecho,ao,mes,sexo,edad,tipopersona,calidadjuridica,clasificaciondelito,lon,lat,geopoint
0,0,8511202,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,2019-08-31 20:30:00,2019.0,Agosto,Femenino,24.0,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,-99.165831,19.265853,"19.2658532425,-99.1658309176"
1,1,8511203,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,2019-08-31 17:00:00,2019.0,Agosto,Femenino,45.0,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,-99.231585,19.358567,"19.3585668959,-99.2315845002"
2,2,8511210,NARCOMENUDEO POSESION SIMPLE,DELITO DE BAJO IMPACTO,2019-08-31 23:30:00,2019.0,Agosto,NaN,NaN,MORAL,VICTIMA,FUERO COMUN,-99.041738,19.379067,"19.3790668024,-99.0417384999"
3,3,8511220,ROBO DE OBJETOS DEL INTERIOR DE UN VEHICULO,DELITO DE BAJO IMPACTO,2019-09-01 00:00:00,2019.0,Septiembre,Femenino,40.0,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,-99.156856,19.335144,"19.3351437,-99.1568559003"
4,4,8511227,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,2019-09-01 01:50:00,2019.0,Septiembre,Femenino,NaN,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,-99.161371,19.317343,"19.3173431639,-99.1613711149"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145865,145865,8533929,ABUSO SEXUAL,DELITO DE BAJO IMPACTO,2019-09-28 20:00:00,2019.0,Septiembre,Femenino,22.0,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,-99.202481,19.372614,"19.3726139581,-99.2024805395"
145866,145866,8533931,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,2019-09-30 20:00:00,2019.0,Septiembre,Femenino,20.0,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,-99.184197,19.397102,"19.3971023956,-99.1841971002"
145867,145867,8533933,DESPOJO,DELITO DE BAJO IMPACTO,2019-08-29 14:00:00,2019.0,Agosto,Femenino,NaN,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,-99.148761,19.311488,"19.3114877811,-99.1487612589"
145868,145868,8533947,HOMICIDIO POR ARMA DE FUEGO,HOMICIDIO DOLOSO,2019-09-30 23:49:00,2019.0,Septiembre,Masculino,30.0,FISICA,CADAVER,FUERO COMUN,-99.144678,19.480919,"19.4809194031,-99.1446780997"


In [3]:
delitos_de_genero = ['VIOLENCIA FAMILIAR','ABUSO SEXUAL','ACOSO SEXUAL','FEMINICIDIO','VIOLACION', 'VIOLACION EQUIPARADA Y ROBO DE VEHICULO','VIOLACION EQUIPARADA', 'VIOLACION TUMULTUARIA','VIOLACION EQUIPARADA POR CONOCIDO','VIOLACION TUMULTUARIA EQUIPARADA','VIOLACION TUMULTUARIA EQUIPARADA POR CONOCIDO']
ubicaciones = []
for i in datos.values:
    if (i[2] in delitos_de_genero) and (not np.isnan(i[12])) and (not np.isnan(i[13])) and (i[7]=='Femenino') and (i[6]=='Agosto'):
        ubicaciones.append((i[13],i[12]))

ubicaciones=np.array(ubicaciones)

In [4]:
datos_limpios=pd.DataFrame(ubicaciones,columns=['lat','long'])
datos_limpios

,lat,long
0,19.265853,-99.165831
1,19.358567,-99.231585
2,19.326496,-99.104204
3,19.295514,-99.006113
4,19.327308,-98.976596
...,...,...
1651,19.376493,-99.052230
1652,19.218645,-98.991242
1653,19.310534,-99.152652
1654,19.465932,-99.188952


In [8]:
datos_limpios.to_csv('datos_limpios.csv',index=False)

# Correr a partir de aqui

In [15]:
import pandas as pd 
import numpy as np 
import networkx as nx

In [16]:
ubicaciones=pd.read_csv('datos_limpios.csv').values.tolist()
ubicaciones[:10]

[[19.2658532425216, -99.1658309175925],
 [19.358566895857606, -99.2315845002375],
 [19.3264961005641, -99.104204400184],
 [19.295514423566, -99.006112507062],
 [19.327307583794997, -98.9765958103246],
 [19.1902780758593, -99.02311049000791],
 [19.463360611004802, -99.19486492544428],
 [19.384031719094, -99.0344912776556],
 [19.483406833861398, -99.1883643146307],
 [19.505770708382197, -99.08805631229349]]

### Construccion del grafo

In [ ]:
from math import sin, cos, sqrt, atan2, radians
R = 6373.0

In [ ]:
G=np.zeros((len(ubicaciones),len(ubicaciones)))
d=np.zeros(len(ubicaciones))
for i in range(len(ubicaciones)):
    for j in range(i+1,len(ubicaciones)):
        dlon = radians(ubicaciones[i][1]) - radians(ubicaciones[j][1])
        dlat = radians(ubicaciones[i][0]) - radians(ubicaciones[j][0])
        a = sin(dlat / 2)**2 + cos(ubicaciones[i][0]) * cos(ubicaciones[j][0]) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        if distance<2:
            G[i,j]=1
            G[j,i]=1
            d[i]+=1
            d[j]+=1

In [ ]:
G

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Buscando los cliques

In [ ]:
import numpy as np 
cliques=[]
  
def is_clique(b) :  
    for i in range(1, b): 
        for j in range(i + 1, b):
            if (G[posible_clique[i]][posible_clique[j]] == 0) : 
                return False;  
    return True;    

def findCliques(G,i, l, s) :
    global posible_clique  
    for j in range(i, len(ubicaciones) -(s - l)) :  
        if (d[j] >= s - 1) : 
            posible_clique[l] = j;  
            if (is_clique(l + 1)) :  
                if (l < s) : 
                    findCliques(G,j, l + 1, s)  
                else : 
                    clique=[]
                    for a in range(1,l+1):
                        clique.append(posible_clique[a])
                        G[posible_clique[a]]=np.zeros(len(ubicaciones))
                        G[:,posible_clique[a]]=np.zeros(len(ubicaciones))
                        d[posible_clique[a]]=0
                    cliques.append(clique)  
                    print(clique)

if __name__ == "__main__" :  
    for k in range(20,9,-1):
        posible_clique = [0]* 20
        findCliques(G,0, 1, k)

[6, 102, 184, 214, 240, 297, 376, 392, 499, 584, 733, 775, 887, 959, 1123, 1364, 1386, 1515, 1630, 1654]
[8, 233, 400, 420, 460, 669, 798, 825, 840, 849, 948, 951, 980, 1020, 1091, 1139, 1225, 1231, 1279, 1300]
[10, 17, 48, 55, 72, 144, 177, 181, 198, 210, 303, 488, 657, 659, 725, 746, 755, 813, 879, 991]
[12, 129, 204, 208, 258, 323, 438, 445, 451, 575, 576, 596, 943, 990, 1043, 1070, 1076, 1078, 1099, 1598]
[15, 58, 65, 86, 188, 191, 274, 331, 345, 525, 602, 736, 836, 1182, 1183, 1257, 1470, 1525, 1561, 1563]
[27, 170, 176, 355, 390, 427, 452, 582, 601, 666, 682, 743, 767, 805, 863, 917, 1095, 1352, 1353, 1363]
[28, 94, 114, 115, 116, 157, 171, 202, 339, 502, 591, 597, 638, 655, 658, 664, 737, 774, 962, 970]
[31, 73, 77, 92, 257, 354, 359, 364, 435, 608, 647, 816, 822, 986, 1071, 1072, 1204, 1220, 1291, 1370]
[33, 122, 217, 218, 302, 494, 497, 614, 622, 826, 864, 925, 1109, 1114, 1332, 1333, 1375, 1387, 1454, 1492]
[35, 134, 159, 163, 267, 283, 501, 739, 748, 851, 870, 871, 927, 966,

# Hayando los centroides a partir de los cliques encontrados

In [ ]:
from math import cos, sin, atan2, sqrt,pi

def centroide(geolocalizaciones):
    lat_centro=0
    long_cos=0
    long_sin=0
    for lat,long in geolocalizaciones:
        lat = float(lat)*pi/180
        long = float(long)*pi/180
        long_cos += cos(long)
        long_sin += sin(long)
        lat_centro += lat
    long_cos /= len(geolocalizaciones)
    long_sin /= len(geolocalizaciones)
    lat_centro /= len(geolocalizaciones)
    lon_centro = atan2(long_sin,long_cos)
    return lat_centro*180/pi, lon_centro*180/pi

In [ ]:
ubicacionesCentrosAtencion=[]
for clique in cliques:
  ubicacionesCentrosAtencion.append(centroide(ubicaciones[clique]))

In [ ]:
ubicacionesCentrosAtencion

[(19.45707836658183, -99.18789488744797),
 (19.485153942885752, -99.1849860694038),
 (19.44112061511942, -99.13329343845444),
 (19.356900414026107, -99.09045261267744),
 (19.403150609594434, -99.18888674274791),
 (19.48871967348721, -99.07643261545363),
 (19.39097688894184, -99.11201313842908),
 (19.42701901952466, -99.09643732635598),
 (19.341803815391106, -99.00305740959983),
 (19.33407879286028, -99.02964568505169),
 (19.41538930944113, -99.06856737339922),
 (19.420307873654036, -99.14843182329183),
 (19.49112382628595, -99.09873139999861),
 (19.378325422611248, -99.21354024874086),
 (19.323251818076294, -99.15825637011817),
 (19.36195872395061, -99.10559689228508),
 (19.393875337698933, -99.14408946576515),
 (19.418031469836, -99.13025087094825),
 (19.430349806727495, -99.16867815192856),
 (19.39065800607491, -99.08810443059001),
 (19.456198965064537, -99.14260872493911),
 (19.44302211576234, -99.10841582729005),
 (19.444462421018553, -99.15018091140855),
 (19.470795548733328, -99.

# Mapeado de las ubicaciones obtenidas

In [ ]:
!pip install folium

In [ ]:

import folium
m = folium.Map(
    location=[19.42847, -99.12766],
    zoom_start=11,
)
for centro in ubicacionesCentrosAtencion:
  folium.Marker(centro, tooltip=str(centro)).add_to(m)
  folium.Circle(centro, 500, fill=True).add_to(m)

m

# Clustering de los delitos

In [ ]:
import folium
from folium.plugins import MarkerCluster
m = folium.Map(
    location=[19.42847, -99.12766],
    zoom_start=11,
)
marker_cluster = MarkerCluster().add_to(m)
for delito in ubicaciones:
  folium.Marker(
    location=delito,
  ).add_to(marker_cluster)

m